In [4]:
from dask.threaded import get

In [33]:
def add(a, b):
    return a + b


dsk = {
    'x': 3,
    'y': 2,
    'z': (add, 'x', 'y')
}

In [34]:
get(dsk, 'z')

5

In [35]:
def add(a, b):
    return len(a) + len(b)


dsk = {
    'x': 3,
    'y': 2,
    'z': (add, ['x'], ['y', 'y'])
}

In [36]:
get(dsk, 'z')

3

In [69]:
def add(a, b):
    return a + b


dsk = {
    'x': 1,
    'y': 2,
    'z': (add, 'x', 'y'),
    'q': [(sum, ['x', 'y', 'z']), 'z']
}

get(dsk, 'q')

[6, 3]

## Custom Graphs

In [85]:
def load(chunk_name):
    print(f"Loading chunk {chunk_name}")
    return f"chunk --- --- {chunk_name}"


def sanitize(chunk):
    print(f"Sanitizing chunk {chunk}")
    return chunk


def mean(chunks):
    print(f"mean {chunks}")
    return chunks[0]


def save(result):
    print("saving result")

dsk = {
    'load-chunk-1': (load, 'chunk1.dat'),
    'load-chunk-2': (load, 'chunk2.dat'),
    'sanitize-1': (sanitize, 'load-chunk-1'),
    'sanitize-2': (sanitize, 'load-chunk-2'),
    'mean': (mean, ["load-chunk-%d" % i for i in [1, 2]]),
    'save': (save, 'mean')
}

In [110]:
from dask.distributed import Client
client = Client(n_workers=2)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 12,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59079,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:59086,Total threads: 6
Dashboard: http://127.0.0.1:59090/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:59082,


2024-10-21 08:13:44,417 - distributed.scheduler - WARNING - Worker failed to heartbeat for 3656s; attempting restart: <WorkerState 'tcp://127.0.0.1:59086', name: 0, status: running, memory: 0, processing: 0>
2024-10-21 08:13:44,622 - distributed.scheduler - WARNING - Worker failed to heartbeat for 3656s; attempting restart: <WorkerState 'tcp://127.0.0.1:59087', name: 1, status: running, memory: 0, processing: 0>
2024-10-21 08:13:45,460 - distributed.nanny - WARNING - Restarting worker
2024-10-21 08:13:45,475 - distributed.nanny - WARNING - Restarting worker
2024-10-21 10:37:54,647 - distributed.nanny - WARNING - Restarting worker
2024-10-21 10:37:54,656 - distributed.nanny - WARNING - Restarting worker
2024-10-21 10:37:54,734 - distributed.nanny - WARNING - Restarting worker
2024-10-21 10:37:54,856 - distributed.nanny - WARNING - Restarting worker
2024-10-21 10:37:54,860 - distributed.nanny - WARNING - Restarting worker
2024-10-21 10:37:55,186 - distributed.nanny - WARNING - Restarting

In [137]:
client.get(dsk, 'save')